# Text-to-MongoDB with QLoRA

This notebook demonstrates a fine-tuned Qwen2.5-Coder-7B model that translates natural language questions into MongoDB `find` and `aggregate` queries.

The base model achieves ~40% accuracy zero-shot. After fine-tuning on ~1,300 synthetic examples using QLoRA (4-bit quantization + LoRA adapters), accuracy jumps to **98.9% on collection schemas it never saw during training**.

- Dataset: [jmorenas/text-to-mongo-dataset-qlora](https://huggingface.co/datasets/jmorenas/text-to-mongo-dataset-qlora)
- Adapter: [jmorenas/text-to-mongo-qlora](https://huggingface.co/jmorenas/text-to-mongo-qlora)

> **Requirements**: A GPU runtime (T4 or better). In Colab: Runtime → Change runtime type → T4 GPU.

## 1. Install Dependencies

In [ ]:
!pip install -q torch transformers peft bitsandbytes accelerate huggingface_hub

## 2. Explore the Dataset

The training data is fully synthetic — no human labeling. Each example contains:
- **schema**: Collection name, domain, and fields with typed semantic roles
- **allowed_ops**: Which MongoDB operators the model is allowed to use
- **intent**: Natural language question
- **output**: Ground-truth MongoDB query

In [ ]:
import json
from huggingface_hub import hf_hub_download

REPO = "jmorenas/text-to-mongo-dataset-qlora"

def load_jsonl(split: str) -> list[dict]:
    """Load a JSONL split from HuggingFace, parsing each line as raw JSON."""
    path = hf_hub_download(repo_id=REPO, filename=f"{split}.jsonl", repo_type="dataset")
    examples = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if line:
                examples.append(json.loads(line))
    return examples

ds = {
    "train": load_jsonl("train"),
    "eval": load_jsonl("eval"),
    "held_out": load_jsonl("held_out"),
}

for split, examples in ds.items():
    print(f"  {split}: {len(examples)} examples")

In [ ]:
# Look at a training example
example = ds["train"][0]
print("Schema:", json.dumps(example["schema"], indent=2))
print("\nAllowed ops:", json.dumps(example["allowed_ops"], indent=2))
print("\nIntent:", example["intent"])
print("\nExpected output:", json.dumps(example["output"], indent=2))

In [ ]:
# Distribution of query types
from collections import Counter

for split in ["train", "eval", "held_out"]:
    types = Counter(ex["output"].get("type", "unknown") for ex in ds[split])
    negatives = sum(1 for ex in ds[split] if ex["is_negative"])
    print(f"{split}: {dict(types)}, negatives: {negatives}")

## 3. Load the Model

We load the base model (Qwen2.5-Coder-7B-Instruct) in 4-bit precision, then apply the LoRA adapter on top.

**Important**: The adapter cannot be merged into 4-bit weights — `merge_and_unload()` silently produces garbage. We keep it as a `PeftModel` wrapper.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen2.5-Coder-7B-Instruct"
ADAPTER = "jmorenas/text-to-mongo-qlora"

# 4-bit quantization config (same as training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print("Loading base model...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, ADAPTER)
model.eval()

print(f"Model loaded on {next(model.parameters()).device}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1e9:.1f} GB")

## 4. Build Prompts

The model uses ChatML format. The system message instructs it to generate MongoDB queries. The user message contains the schema, allowed operators, and natural language intent.

> **Important**: Field descriptions must be short (2-5 words). The model was trained on concise descriptions like `"Order total"` or `"User email"`. Longer descriptions (e.g. `"The total monetary value of the order including tax and shipping"`) cause the model to hallucinate operator lists instead of generating queries. Keep descriptions brief — the semantic role (`measure`, `enum`, `timestamp`, etc.) already carries most of the meaning.

In [ ]:
SYSTEM_PROMPT = (
    "You are a MongoDB query generator. Given a collection schema, a list of "
    "allowed operators, and a natural language intent, produce a valid MongoDB "
    "query as JSON. The output must be a JSON object with a 'type' field "
    "('aggregate' or 'find') and the corresponding query body. "
    "If the intent references fields not in the schema, respond with "
    '{"error": "<reason>"}. '
    "Use only the allowed operators."
)


def build_prompt(schema: dict, allowed_ops: dict, intent: str) -> str:
    """Build a ChatML prompt from schema, allowed ops, and intent."""
    # Render schema
    lines = [f"Collection: {schema['collection']}"]
    lines.append("Fields:")
    for f in schema["fields"]:
        parts = [f"  - {f['name']} ({f['type']}, {f['role']})"]
        if f.get("description"):
            parts.append(f": {f['description']}")
        if f.get("enum_values"):
            parts.append(f" [values: {', '.join(f['enum_values'])}]")
        lines.append("".join(parts))

    # Render operators
    lines.append("")
    lines.append("Allowed stage operators: " + ", ".join(allowed_ops["stage_operators"]))
    lines.append("Allowed expression operators: " + ", ".join(allowed_ops["expression_operators"]))
    lines.append("")
    lines.append(f"Intent: {intent}")

    user_msg = "\n".join(lines)

    return (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n{user_msg}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )


# Show what a prompt looks like
sample = ds["eval"][0]
prompt = build_prompt(sample["schema"], sample["allowed_ops"], sample["intent"])
print(prompt)

## 5. Run Inference

Let's generate queries from examples in the dataset and compare against the expected output.

In [ ]:
def generate_query(schema: dict, allowed_ops: dict, intent: str,
                   target_model=None, max_new_tokens: int = 256) -> str:
    """Generate a MongoDB query from natural language."""
    m = target_model or model
    prompt = build_prompt(schema, allowed_ops, intent)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(m.device)

    with torch.no_grad():
        output_ids = m.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )

    # Decode only the generated tokens (skip the prompt)
    prompt_len = inputs["input_ids"].shape[1]
    generated = output_ids[0][prompt_len:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()

In [ ]:
# Test on a few eval examples
for i in range(5):
    ex = ds["eval"][i]
    output = generate_query(ex["schema"], ex["allowed_ops"], ex["intent"])

    print(f"--- Example {i+1} ---")
    print(f"Collection: {ex['schema']['collection']}")
    print(f"Intent: {ex['intent']}")
    print(f"Expected: {json.dumps(ex['output'])}")
    print(f"Got:      {output}")

    try:
        match = json.loads(output) == ex["output"]
        print(f"Match: {'yes' if match else 'no'}")
    except json.JSONDecodeError:
        print("Match: no (invalid JSON)")
    print()

## 6. Generalization — Unseen Schemas

The real test: can the model generate correct queries for collection schemas it **never saw during training**?

The held-out set contains 3 collections (`museum_exhibits`, `weather_stations`, `fleet_vehicles`) that were excluded from training entirely.

In [ ]:
# Test on held-out examples (unseen schemas)
held_out_collections = set()
for i in range(5):
    ex = ds["held_out"][i]
    held_out_collections.add(ex["schema"]["collection"])
    output = generate_query(ex["schema"], ex["allowed_ops"], ex["intent"])

    print(f"--- Held-out {i+1} ---")
    print(f"Collection: {ex['schema']['collection']} (NEVER seen in training)")
    print(f"Intent: {ex['intent']}")
    print(f"Expected: {json.dumps(ex['output'])}")
    print(f"Got:      {output}")

    try:
        match = json.loads(output) == ex["output"]
        print(f"Match: {'yes' if match else 'no'}")
    except json.JSONDecodeError:
        print("Match: no (invalid JSON)")
    print()

print(f"Held-out collections tested: {held_out_collections}")

## 7. Try Your Own Schemas

The model has never seen these schemas — it reads the field names, types, and roles at inference time to compose queries.

In [ ]:
# Define a custom schema the model has never seen
custom_schema = {
    "collection": "books",
    "domain": "library",
    "fields": [
        {"name": "isbn", "type": "string", "role": "identifier", "description": "Book ISBN"},
        {"name": "title", "type": "string", "role": "text", "description": "Book title"},
        {"name": "genre", "type": "string", "role": "enum", "description": "Book genre",
         "enum_values": ["fiction", "non-fiction", "science", "history", "biography"]},
        {"name": "pages", "type": "int", "role": "measure", "description": "Page count"},
        {"name": "rating", "type": "double", "role": "measure", "description": "Average rating"},
        {"name": "published_at", "type": "date", "role": "timestamp", "description": "Publication date"},
        {"name": "available", "type": "bool", "role": "boolean", "description": "In stock"},
    ],
}

custom_ops = {
    "stage_operators": ["$match", "$group", "$sort", "$limit", "$project"],
    "expression_operators": ["$sum", "$avg", "$gt", "$gte", "$lt", "$eq", "$in"],
}

queries = [
    "Find all available science books",
    "What is the average rating per genre?",
    "Show the top 5 highest-rated fiction books",
    "How many books are in each genre?",
    "Find books published after January 2024 with more than 300 pages",
]

for intent in queries:
    output = generate_query(custom_schema, custom_ops, intent)
    print(f"Q: {intent}")
    try:
        parsed = json.loads(output)
        print(f"A: {json.dumps(parsed, indent=2)}")
    except json.JSONDecodeError:
        print(f"A (raw): {output}")
    print()

### CI/CD Dashboard — Real-world use case

This is the schema from a real CI/CD dashboard that tracks container artifacts, product drops, and build pipelines. The model was never trained on any of these collections.

In [ ]:
# CI/CD Dashboard schemas — artifacts, drops, products
artifacts_schema = {
    "collection": "artifacts",
    "domain": "cicd_dashboard",
    "fields": [
        {"name": "key", "type": "string", "role": "identifier", "description": "Artifact ID"},
        {"name": "type", "type": "string", "role": "enum", "description": "Artifact type",
         "enum_values": ["containers", "disk-images", "cloud-disk-images",
                         "disk-image-containers", "cloud-containers",
                         "base-images", "wheels-collections", "instructlab",
                         "models", "model-cars"]},
        {"name": "product_key", "type": "string", "role": "enum", "description": "Product key",
         "enum_values": ["rhel-ai", "rhaiis", "base-images", "builder-images"]},
        {"name": "variant", "type": "string", "role": "category", "description": "Accelerator+OS combo"},
        {"name": "archs", "type": "array", "role": "category", "description": "CPU architectures",
         "enum_values": ["x86_64", "aarch64", "s390x", "ppc64le"]},
        {"name": "created_at", "type": "date", "role": "timestamp", "description": "Build timestamp"},
        {"name": "environments", "type": "array", "role": "enum", "description": "Deploy environments",
         "enum_values": ["stage", "production"]},
        {"name": "series", "type": "string", "role": "category", "description": "Version series"},
    ],
}

drops_schema = {
    "collection": "drops",
    "domain": "cicd_dashboard",
    "fields": [
        {"name": "key", "type": "string", "role": "identifier", "description": "Drop key"},
        {"name": "name", "type": "string", "role": "text", "description": "Drop version name"},
        {"name": "product_key", "type": "string", "role": "enum", "description": "Product key",
         "enum_values": ["rhel-ai", "rhaiis", "base-images", "builder-images"]},
        {"name": "product_version", "type": "string", "role": "text", "description": "Semantic version"},
        {"name": "created_at", "type": "date", "role": "timestamp", "description": "Creation time"},
        {"name": "announced_at", "type": "date", "role": "timestamp", "description": "Announcement time"},
        {"name": "published_at", "type": "date", "role": "timestamp", "description": "Publication time"},
    ],
}

cicd_ops = {
    "stage_operators": ["$match", "$group", "$sort", "$limit", "$project", "$unwind", "$count"],
    "expression_operators": [
        "$sum", "$avg", "$min", "$max", "$first", "$last",
        "$eq", "$ne", "$gt", "$gte", "$lt", "$lte",
        "$in", "$nin", "$exists", "$regex",
        "$and", "$or",
        "$year", "$month", "$dayOfMonth",
    ],
}

print("=== Artifacts Collection ===\n")
for intent in [
    "Show the latest rhaiis containers",
    "How many artifacts per product?",
    "Find all disk-images built for aarch64",
    "Count artifacts by type",
]:
    output = generate_query(artifacts_schema, cicd_ops, intent)
    print(f"Q: {intent}")
    try:
        print(f"A: {json.dumps(json.loads(output), indent=2)}")
    except json.JSONDecodeError:
        print(f"A (raw): {output}")
    print()

print("=== Drops Collection ===\n")
for intent in [
    "Show all rhel-ai drops sorted by creation date",
    "How many drops per product?",
    "Find drops announced after January 2025",
]:
    output = generate_query(drops_schema, cicd_ops, intent)
    print(f"Q: {intent}")
    try:
        print(f"A: {json.dumps(json.loads(output), indent=2)}")
    except json.JSONDecodeError:
        print(f"A (raw): {output}")
    print()

## 8. Evaluate Accuracy

Run the full eval and held-out sets through the model and measure syntax validity and exact match rates.

In [ ]:
from time import time

def evaluate_split(split_name: str, max_examples: int = 50):
    """Evaluate model accuracy on a dataset split."""
    examples = ds[split_name]
    n = min(len(examples), max_examples)

    syntax_ok = 0
    exact_match = 0
    total_time = 0

    for i in range(n):
        ex = examples[i]
        start = time()
        output = generate_query(ex["schema"], ex["allowed_ops"], ex["intent"])
        total_time += time() - start

        try:
            parsed = json.loads(output)
            syntax_ok += 1
            if parsed == ex["output"]:
                exact_match += 1
        except json.JSONDecodeError:
            pass

        if (i + 1) % 10 == 0:
            print(f"  {i+1}/{n} done...")

    avg_latency = total_time / n
    print(f"\n{split_name} ({n} examples):")
    print(f"  Syntax valid: {syntax_ok}/{n} ({100*syntax_ok/n:.1f}%)")
    print(f"  Exact match:  {exact_match}/{n} ({100*exact_match/n:.1f}%)")
    print(f"  Avg latency:  {avg_latency:.2f}s")

# Evaluate on first 50 examples from each split
# (set max_examples higher for full evaluation)
evaluate_split("eval", max_examples=50)
evaluate_split("held_out", max_examples=50)

## How It Works

The model was fine-tuned using **QLoRA** — a parameter-efficient method that:

1. **Quantizes** the base model to 4-bit (NF4), reducing memory from ~14GB to ~4GB
2. **Adds small trainable adapters** (LoRA, rank 8) to attention and MLP layers — only ~0.1% of parameters are trained
3. **Trains on synthetic data** — 19 MongoDB schemas × 10 query patterns × augmentation = ~1,300 examples
4. **Masks prompt tokens** — the model only learns to generate the JSON query, not to repeat the schema

The key insight: each field in the schema has a **semantic role** (`identifier`, `measure`, `timestamp`, `category`, `enum`, `boolean`, `text`) that tells the model how to use it. A `measure` field gets summed/averaged in `$group`, a `timestamp` gets range-filtered with `$gte`/`$lte`, an `enum` gets filtered with `$in`. The model learned to read these roles and compose queries accordingly — even for schemas it has never seen.

For more details, see the [GitHub repository](https://github.com/jangel97/text-to-mongo).

## 9. Baseline Comparison — Before vs After

What does the base model produce **without** the LoRA adapter? Let's disable the adapter and run the same examples to see the difference fine-tuning makes.

In [ ]:
# Disable the LoRA adapter to get the base model's zero-shot output
model.disable_adapter_layers()

# Pick examples from both eval and held-out splits
comparison_examples = [ds["eval"][0], ds["eval"][5], ds["eval"][10], ds["held_out"][0], ds["held_out"][5]]

print("=" * 70)
print("BASELINE (zero-shot, no adapter)")
print("=" * 70)

baseline_matches = 0
for i, ex in enumerate(comparison_examples):
    output = generate_query(ex["schema"], ex["allowed_ops"], ex["intent"])
    print(f"\n--- Example {i+1}: {ex['schema']['collection']} ---")
    print(f"Intent:   {ex['intent']}")
    print(f"Expected: {json.dumps(ex['output'])}")
    print(f"Got:      {output}")
    try:
        if json.loads(output) == ex["output"]:
            baseline_matches += 1
            print("Match: yes")
        else:
            print("Match: no")
    except json.JSONDecodeError:
        print("Match: no (invalid JSON)")

# Re-enable the LoRA adapter
model.enable_adapter_layers()

print("\n" + "=" * 70)
print("FINE-TUNED (with LoRA adapter)")
print("=" * 70)

finetuned_matches = 0
for i, ex in enumerate(comparison_examples):
    output = generate_query(ex["schema"], ex["allowed_ops"], ex["intent"])
    print(f"\n--- Example {i+1}: {ex['schema']['collection']} ---")
    print(f"Intent:   {ex['intent']}")
    print(f"Expected: {json.dumps(ex['output'])}")
    print(f"Got:      {output}")
    try:
        if json.loads(output) == ex["output"]:
            finetuned_matches += 1
            print("Match: yes")
        else:
            print("Match: no")
    except json.JSONDecodeError:
        print("Match: no (invalid JSON)")

n = len(comparison_examples)
print(f"\n{'=' * 70}")
print(f"Baseline:   {baseline_matches}/{n} correct ({100*baseline_matches/n:.0f}%)")
print(f"Fine-tuned: {finetuned_matches}/{n} correct ({100*finetuned_matches/n:.0f}%)")
print(f"{'=' * 70}")

## Conclusion

The baseline isn't dumb — it's undisciplined. Look at the baseline outputs: the model *understands* MongoDB. It picks the right operators, references the right fields, builds logically correct queries. But it fails on three things the fine-tuning drills in:

1. **Output format** — The baseline wraps everything in `` ```json `` markdown fences instead of raw JSON. That alone makes every single output unparseable by downstream code.

2. **Schema compliance** — It uses `"query"` instead of `"filter"`, picks operators outside the allowed list (e.g. `$dateToString` instead of `$dayOfMonth`), and invents its own alias names (`min_session_length`, `max_price`) instead of following the training format. The base model draws from its general MongoDB knowledge rather than reading the specific prompt constraints.

3. **Date format** — It uses JavaScript `new Date()` constructors instead of Extended JSON `{"$date": "..."}`. Correct in a mongo shell, but not parseable as JSON.

The fine-tuned model produces **byte-identical output** to expected — not just structurally correct, but exact matches. It learned the precise output contract: raw JSON, `filter` not `query`, Extended JSON dates, specific naming conventions.

The held-out examples (`museum_exhibits`, `weather_stations`, `fleet_vehicles`) prove the model learned to **read schemas** rather than memorize collection-specific patterns.

**The takeaway**: we didn't teach the model MongoDB — it already knew MongoDB. We taught it to follow a strict output protocol, and that's exactly the kind of narrow behavioral constraint where LoRA shines over prompt engineering.